USING THIS AS BASE OF OUR MATCHING: https://www.datacamp.com/community/tutorials/fuzzy-string-python

# IMPORTS

In [1]:
import yaml
import firebase_admin
import plotly
import plotly.plotly as py
import numpy as np
import re

import pandas as pd

import geocoder


import gspread
import httplib2
import os, io
import argparse
import auth

import unidecode


from firebase_admin import credentials
from firebase_admin import firestore


from httplib2 import Http
from oauth2client import file,client, tools
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials



from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# CONNECT TO FIREBASE

In [2]:
filename='/Users/5danalfaro/OAN/credentials/firebase_parameters.json'

In [3]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## TEST

In [4]:
col_query = db.collection('Pompes_2019').where(u'Arrondissement', u'==', u'BIRO')

In [5]:
pumps = col_query.stream()

In [7]:
#for pump in pumps:
#    print(u'{} => {}'.format(pump.id, pump.to_dict()))

In [210]:
col= db.collection('Locations')

In [211]:
doc = col.document('Benin')

In [13]:
get_doc=doc.get()

In [14]:
print(u'{} => {}'.format(get_doc.id,get_doc.to_dict()))

Benin => {'population': 11180000, 'id': 'Benin', 'coordinates': {'longitud': '2.315834', 'latitud': '9.30769'}}


# ABRIR LAS BBDD

## OPEN GOOGLE SHEETS

## AUTHENTIFICATION

In [4]:
SCOPES = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/spreadsheets.currentonly',
         'https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/presentations',
         'https://www.googleapis.com/auth/drive.scripts',
         'https://www.googleapis.com/auth/drive.appdata',
         'https://www.googleapis.com/auth/drive.metadata'
        ]
store = file.Storage('/Users/5danalfaro/OAN/credentials/storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('/Users/5danalfaro/OAN/credentials/client_id.json', SCOPES)
    creds = tools.run_flow(flow, store)
drive_service = discovery.build('drive', 'v3', http=creds.authorize(Http()))
sheet_service = discovery.build('sheets', 'v4', http=creds.authorize(Http()))

## GOOGLE SHETS IDS

In [5]:
spreadsheet_ids = yaml.load( open("/Users/5danalfaro/OAN/credentials/sheets_parameters.yaml", "r" ),Loader=yaml.FullLoader)
#bombas de 2018, ultima version
ID1=str(spreadsheet_ids['ID1'])
#bombas de 2016, ultima version
ID2=str(spreadsheet_ids['ID2'])
#bombas de 2009, ultima version
ID3=str(spreadsheet_ids['ID3'])
#bombas final, datos de 2016 y 2009
ID4=str(spreadsheet_ids['ID4'])
#bombas lisette
ID5=str(spreadsheet_ids['ID5'])
#demograpique data
ID6=str(spreadsheet_ids['ID6'])

## get sheets

In [716]:
ID5

'1WYGvBKMxdpwclamhp0VPb_WguyTx2b-rJsjnvdwl5-k'

In [17]:
ids = [ID1,ID2,ID3,ID5]
dfs = []
for ID in ids:
    #SPREADSHEET_ID = ID
    RANGE_NAME = 'Sheet1'
    
    gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID, range=RANGE_NAME).execute()
    
    dfs.append(pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0]))

df_2019=dfs[0]
df_2016=dfs[1]
df_2009=dfs[2]
df_lisette = dfs[3]

In [18]:
dfs[1]["Etat de l'ouvrage"].unique()

array(['Fonctionnel', 'Panne', 'En panne', 'Tête de pompe', '2 pompes',
       'Privé', '1  Pompe', '', 'Pompe', 'Pompe non', 'Faible débit',
       '1 pompe', 'Kalale'], dtype=object)

In [19]:
[df.columns for df in dfs]
#df_2016.head(2)
#df_2009.head(2)

[Index(['Arrondissement', 'Village', 'Localite', 'Nom_Local', 'Longitude',
        'Latitude', 'xCoord', 'yCoord', 'rqtListe_Pompe.Modele_Pompe'],
       dtype='object'),
 Index(['N°', 'Arrondissement', 'Village', 'Localité', 'Hameau',
        'Etat de l'ouvrage', 'Redevance', 'Année de réalisation', 'Marque'],
       dtype='object'),
 Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
        'Etat', 'Longitude', 'Latitude', 'Domaine d'installation'],
       dtype='object'),
 Index(['Departement', 'Commune', 'Arrondissement', 'Village', 'Localite',
        'Nom_Local', 'Longitude', 'Latitude', 'xCoord', 'yCoord',
        'Type_Exhaure', 'Modele_Pompe'],
       dtype='object')]

So common data is:

* 2019 vs 2016: Arrondissement, Village, Localite, Marque

* 2019 vs 2009: Arrondissement, Village, Localite, Longitude, Latitude

* 2009 vs 2016: Arrondissement, Village, Localite, Etat

ONLY DATA:

* 2019: Nom_Local which is little identifier for more info on pumps

* 2016: Année de réalisation, , Redevace, Hameau, which would be kind of the same as Nom_Local

* 2009: Population, Type d'ouvrage

MISSING:

* 2019: Population, other villages, Année de réalisation, , Redevace, Hameau

* 2016: Population, other villages

* 2009: Année de réalisation, Redevace, Hameau

### LETS CLEAN THE DATA

#### Functions

In [20]:
#Marques des Pompes
def marques(x,name,name2=0):
    marque='NA'
  
    if x[name]:
        if bool(re.search('ergnet',x[name].lower())):
            marque='Vergnet'
        elif bool(re.search('ndia',x[name].lower())):
            marque='India'
        elif bool(re.search('afri',x[name].lower())):
            marque='Afridev'
    elif x[name2]:
        if bool(re.search('moteur',x[name2].lower())):
            marque='pompe_a_moteur'
    return marque

In [36]:
#Years
def getyear(x,col):
    year=None
    if x[col]:
        for s in x[col].split():
            if s.isdigit():
                year=int(s)
        if year==None:
            return year
    elif year >2019:
        #print(year)
        year=2010
        return year
    elif year > 100:
        return int(year)
    elif year >70:
        year=int('19'+str(year))
        return int(year)
    else:
        year=None
    
    return year

In [23]:
'''
Function to change latitude/longitude data from degrees,minutes and seconds into decimals
'''
def decimals_lat_lon(x,lat):
    coord=None
    if x[lat]:
        if len(x[lat])>2:
            if len(re.split('[°\'"]', x[lat])) == 4 :
        
                #gets all the parts of the coordinates into different elements of a vector
                deg, minutes, seconds, direction =  re.split('[°\'"]', x[lat])

                #had to put replace commas because of data of 2009
                coord=(float(deg) + float(minutes.replace(",","."))/60 + float(seconds.replace(",","."))/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
            elif len(re.split('[°\'"]', x[lat])) > 0:
                coord=x[lat]
                print(coord)
    else:
        coord=None
        print(coord)
    return coord

# 2009

2009 localite source of truth

In [24]:
df_2009.columns

Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
       'Etat', 'Longitude', 'Latitude', 'Domaine d'installation'],
      dtype='object')

In [25]:
df_2009.columns = [unidecode.unidecode(a.lower()) for a in df_2009.columns]

In [26]:
df_2009.columns

Index(['arrondissement', 'village', 'localite', 'population', 'type d'ouvrage',
       'etat', 'longitude', 'latitude', 'domaine d'installation'],
      dtype='object')

In [34]:
df_2009.head()

,arrondissement,village,localite,population,type d'ouvrage,etat,longitude,latitude,domaine d'installation,latitude_decimals,longitude_decimals
0,BIRO,Biro,Bah Boré,150.0,/,/,/,/,/,NaN,NaN
1,BIRO,Biro,Biro,5000.0,FPM,Fonctionnel,"02°56'48,5""E","09°54'10,7""N",Marché,9.902972,2.946806
2,BIRO,Biro,Biro,5000.0,FPM,Fonctionnel,"02°56'52,0""E","09°54'18,6""N",Domaine public,9.905167,2.947778
3,BIRO,Biro,Biro,5000.0,FPM,Fonctionnel,"02°57'03,7""E","09°54'11,4""N",Domaine public,9.903167,2.951028
4,BIRO,Biro,Biro,5000.0,PM,Fonctionnel,"02°56'51,1""E","09°54'08,5""N",Marché,9.902361,2.947528


In [43]:
df_2009.columns=['arrondissement', 'village', 'localite', 'population', 'type',
       'etat', 'longitude', 'latitude', 'domaine','latitude_decimals','longitude_decimals']

In [27]:
df_2009['latitude_decimals']=df_2009.apply(lambda x: decimals_lat_lon(x,'latitude'),axis=1)
df_2009['longitude_decimals']=df_2009.apply(lambda x: decimals_lat_lon(x,'longitude'),axis=1)

None
None
None
None
None
None
None
None
None
None


In [29]:
df_2009.population=pd.to_numeric(df_2009.population, errors='coerce')

In [44]:
df_2009_tomerge=df_2009[['arrondissement', 'village', 'localite', 'type',
                         'population','latitude_decimals','longitude_decimals']]

# 2016

In [33]:
df_2016.columns

Index(['N°', 'Arrondissement', 'Village', 'Localité', 'Hameau',
       'Etat de l'ouvrage', 'Redevance', 'Année de réalisation', 'Marque'],
      dtype='object')

In [31]:
df_2016.head()

,N°,Arrondissement,Village,Localité,Hameau,Etat de l'ouvrage,Redevance,Année de réalisation,Marque
0,284,BIRO,Biro,Nallou,Nallou Banikparou,Fonctionnel,Jamais,2004,Vergnet
1,285,BIRO,Biro,Goutto,Goutto,Fonctionnel,Jamais,2011,Vergnet
2,286,BIRO,Biro,Nanlou,Nanlou,Fonctionnel,Jamais,,Vergnet
3,287,BIRO,Biro,Sarawodo,Sarawodo,Fonctionnel,Parfait,Entente 95,Vergnet
4,288,BIRO,Biro,Biro,Biro 1,Fonctionnel,Parfait,PPEA 2010,Vergnet


In [35]:
df_2016.columns = [unidecode.unidecode(a.lower()) for a in df_2016.columns]

In [40]:
df_2016.columns = ['n', 'arrondissement', 'village', 'localite', 'hameau',
       'etat', 'redevance', 'annee', 'marque']

In [41]:
df_2016.annee=pd.to_numeric(df_2016.annee,errors='coerce')

In [45]:
df_2016_tomerge=df_2016[['arrondissement', 'village', 'localite', 'hameau',
       'etat', 'redevance', 'annee', 'marque']]

In [59]:
df_2016_tomerge['marque']=df_2016_tomerge.apply(lambda x: marques(x,"marque"),axis=1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
df_2016_tomerge['marque'].value_counts()

Vergnet    224
India       87
NA          40
Afridev      2
Name: marque, dtype: int64

# 2019

In [46]:
#Create decimals from degrees latitude
df_2019['Latitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

None
None


In [47]:
df_2019['Longitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

None
None


In [49]:
df_2019.columns = [unidecode.unidecode(a.lower()) for a in df_2019.columns]

In [51]:
df_2019.columns

Index(['arrondissement', 'village', 'localite', 'nom_local', 'longitude',
       'latitude', 'xcoord', 'ycoord', 'rqtliste_pompe.modele_pompe',
       'latitude_decimals', 'longitude_decimals'],
      dtype='object')

In [63]:
df_2019.columns=['arrondissement', 'village', 'localite', 'nom_local', 'longitude',
       'latitude', 'xcoord', 'ycoord', 'marque',
       'latitude_decimals', 'longitude_decimals']

In [68]:
df_2019_tomerge = df_2019[['arrondissement', 'village', 'localite', 'nom_local', 'marque',
       'latitude_decimals', 'longitude_decimals']]

In [69]:
df_2019_tomerge['marque']=df_2019_tomerge.apply(lambda x: marques(x,"marque",),axis=1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
df_2019_tomerge['marque'].value_counts()

Vergnet    209
NA          85
India       47
Afridev     24
Name: marque, dtype: int64

## Process data of Lisette

In [72]:
df_lisette.head()

,Departement,Commune,Arrondissement,Village,Localite,Nom_Local,Longitude,Latitude,xCoord,yCoord,Type_Exhaure,Modele_Pompe
0,BORGOU,NIKKI,BIRO,BIRO,BIRO NOUKOUROU,,"02°56'50.96""E","09°54'18.50""N",2.947489,9.905139,PMH,VERGNET 4C
1,BORGOU,NIKKI,BIRO,BIRO,GOUTTO,,"02°56'47.38""E","09°54'59.99""N",2.946494,9.916664,PMH,VERGNET 4D
2,BORGOU,NIKKI,BIRO,BIRO,SARAVONDO,,"02°58'29.33""E","09°54'13.30""N",2.974814,9.903694,PMH,VERGNET 4D
3,BORGOU,NIKKI,BIRO,BIRO,MADINA,,"02°56'44.34""E","09°54'02.71""N",2.94565,9.900753,PMH,VERGNET 4C
4,BORGOU,NIKKI,BIRO,BIRO,MADINA,,"02°56'46.83""E","09°54'01.55""N",2.946342,9.900431,PMH,VERGNET 4C


In [73]:
#Create decimals from degrees latitude
df_lisette['Latitude_decimals']=df_lisette.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

In [74]:
df_lisette['Longitude_decimals']=df_lisette.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

In [75]:
df_lisette['Marque']=df_lisette.apply(lambda x: marques(x,"Modele_Pompe","Type_Exhaure"),axis=1)

In [76]:
df_lisette.columns = [unidecode.unidecode(a.lower()) for a in df_lisette.columns]

In [78]:
df_lisette.columns

Index(['departement', 'commune', 'arrondissement', 'village', 'localite',
       'nom_local', 'longitude', 'latitude', 'xcoord', 'ycoord',
       'type_exhaure', 'modele_pompe', 'latitude_decimals',
       'longitude_decimals', 'marque'],
      dtype='object')

In [81]:
df_lisette_tomerge= df_lisette[['arrondissement','village','localite','nom_local','latitude_decimals',
                                'longitude_decimals','marque']]

## VILLAGES

Only Nikki - get villages from work done by FICAI: https://www.google.com/maps/d/edit?mid=1te53KEJYUJ2ulJLsGpa9S15suY1lNPGQ&ll=10.108170603054887%2C2.9732788000000028&z=8

In [82]:
import xml.sax, xml.sax.handler
import re
from zipfile import ZipFile

In [83]:
filename = 'Poblacion.kmz'

kmz = ZipFile(filename, 'r')
kml = kmz.open('doc.kml', 'r')

In [84]:

class PlacemarkHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.inName = False # handle XML parser events
        self.inPlacemark = False
        self.mapping = {} 
        self.buffer = ""
        self.name_tag = ""
        
    def startElement(self, name, attributes):
        if name == "Placemark": # on start Placemark tag
            self.inPlacemark = True
            self.buffer = "" 
        if self.inPlacemark:
            if name == "name": # on start title tag
                self.inName = True # save name text to follow
            
    def characters(self, data):
        if self.inPlacemark: # on text within tag
            self.buffer += data # save text if in title
            
    def endElement(self, name):
        self.buffer = self.buffer.strip('\n\t')
        
        if name == "Placemark":
            self.inPlacemark = False
            self.name_tag = "" #clear current name
        
        elif name == "name" and self.inPlacemark:
            self.inName = False # on end title tag            
            self.name_tag = self.buffer.strip()
            self.mapping[self.name_tag] = {}
        elif self.inPlacemark:
            if name in self.mapping[self.name_tag]:
                self.mapping[self.name_tag][name] += self.buffer
            else:
                self.mapping[self.name_tag][name] = self.buffer
        self.buffer = ""

In [85]:
parser = xml.sax.make_parser()
handler = PlacemarkHandler()
parser.setContentHandler(handler)
parser.parse(kml)
kmz.close()

In [86]:
def build_table(mapping):
    sep = ','
        
    output = []
    shapes = ''
    for key in mapping:
        coord_str = mapping[key]['coordinates']
        coords = re.findall('\d.*,.*\d', coord_str )
        coord = coords[0].replace(',0',"")
        coord = coord.split(',')
        name = key.split(' - ')
        if len(name)==1:
          continue
        else:
          output.append({'Name':name[1],'Arrondissement':name[0],'Longitude':coord[0],'Latitude':coord[1]})
    return output

In [87]:
outstr = build_table(handler.mapping)

In [88]:
df_villages = pd.DataFrame(outstr)

In [89]:
df_villages = df_villages[['Name','Arrondissement','Longitude','Latitude']]

In [90]:
df_villages['Arrondissement'] = df_villages.apply(lambda x: 'GNONKOUROKALI' 
                                                  if bool(re.search('gnonkou',x['Arrondissement'].lower()))
                                                  else str(x['Arrondissement']).upper(), 
                                                  axis=1)

In [91]:
len(df_villages)

84

In [93]:
len(df_lisette['village'].unique())

90

In [96]:
df_villages_lisette =df_lisette.groupby('village')[['localite','longitude_decimals',
                              'latitude_decimals']].apply(lambda group_series: 
                                                          (group_series['localite'].tolist(),
                                                          group_series['longitude_decimals'].tolist(),
                                                          group_series['latitude_decimals'].tolist())).reset_index()


In [97]:
df_villages_lisette['arrondissement']=df_villages_lisette.apply(lambda x: 
                          df_lisette[df_lisette['village']==x['village']]['arrondissement'].values[0],
                         axis=1)

In [98]:
df_villages_lisette['arrondissement'] = df_villages_lisette.apply(lambda x: 'GNONKOUROKALI' 
                                                  if bool(re.search('gnonkou',x['arrondissement'].lower()))
                                                  else str(x['arrondissement']).upper(), 
                                                  axis=1)

In [100]:
l_villages = df_villages_lisette['village'].to_list()
df_villages[['village','match']]=df_villages.apply(lambda x: pd.Series([(a[0],a[1]) for a in process.extract(x['Name'], 
                                                                 l_villages, 
                                                                 limit=1)][0]), axis=1)


In [101]:
df_villages[df_villages['match']<92]

,Name,Arrondissement,Longitude,Latitude,village,match
19,Ouenou,OUENOU,3.2696342,9.9095441,OUENOU -NIKKI,90
21,Ourmon,OUENOU,3.3334923,9.8792314,OUROUMON,86
23,Tchikandou,OUENOU,3.3567309,9.8400999,TCHICANDOU,90
24,Gotel Centre,OUENOU,3.3747768,9.8603745,GOTEL,90
28,Gnel sanna,OUENOU,3.4461451,9.9443769,GNELSANDA,84
29,Goussogui Gouré Baba,OUENOU,3.4822369,9.9212963,GOSSOGUI- GOUREBABA,89
30,Barkedje (Gothel Centre),OUENOU,3.3813429,9.8537362,BARKEDJE,90
36,Serekale Baka,SEREKALE,3.0428481,9.9187388,SEREKALI -BAKA,89
47,Dema,TASSO,3.1871939,9.7339712,DEMAN,89
54,Gbarire,TASSO,3.1944466,9.6748975,GBABIRE,86


In [102]:
reanalysis='Barkedje (Gothel Centre)'
process.extract(reanalysis,l_villages,limit=2)

[('BARKEDJE', 90), ('GOTEL', 72)]

In [103]:
df_villages.loc[30,'Village']='GOTEL'

In [104]:
df_villages[df_villages['Village']=='GOTEL']

,Name,Arrondissement,Longitude,Latitude,village,match,Village
30,Barkedje (Gothel Centre),OUENOU,3.3813429,9.8537362,BARKEDJE,90,GOTEL


In [105]:
[val for val in df_villages['Village'].value_counts().sort_values() if val>1]

[]

In [106]:
#df_villages=df_villages.drop(30)
df_villages=df_villages.reset_index()

In [107]:
df_villages=df_villages.drop('index',axis=1)

In [109]:
df_villages_loc_pumps=pd.merge(df_villages_lisette,df_villages,on='village',how='outer')

In [112]:
def getcoord_loc(x):
    local=x[0]
    l_local=local[0]
    village=x['village']
    
    match=[(a[0]) for a in process.extract(village, 
                                           l_local, 
                                           limit=1)][0]
    long=local[1][l_local.index(match)]
    lat=local[2][l_local.index(match)]
    
    for col,value in zip(['Name','Latitude','Longitude'],[match,lat,long] ):
    
        df_villages_loc_pumps.loc[x.name,col] = value

In [113]:
df_reanalysis = df_villages_loc_pumps[df_villages_loc_pumps['Name'].isna()]
df_reanalysis.apply(lambda x: getcoord_loc(x), axis=1)

9     None
13    None
21    None
50    None
84    None
89    None
90    None
dtype: object

In [116]:
df_villages_loc_pumps.columns

Index([       'village',                0, 'arrondissement',           'Name',
       'Arrondissement',      'Longitude',       'Latitude',          'match',
              'Village'],
      dtype='object')

In [117]:
df_villages_loc_pumps[df_villages_loc_pumps['Arrondissement']!=df_villages_loc_pumps['arrondissement']]

,village,0,arrondissement,Name,Arrondissement,Longitude,Latitude,match,Village
4,BARKEDJE,"([GNEL PEDELE 1, BARIKEDJE], [3.28540833333333...",NIKKI,Barkedje (Gothel Centre),OUENOU,3.3813429,9.8537362,90.0,GOTEL
9,BOO,"([BOOH 1], [3.403436111111111], [9.980075])",NIKKI,BOOH 1,NaN,3.40344,9.98007,NaN,NaN
13,DANNON,"([DANNON, GANDO BAKA], [3.050969444444444, 3.0...",SUYA,DANNON,NaN,3.05097,9.77238,NaN,NaN
21,GAH-GBEROU,"([Bona Soua, Sèkentè], [3.3064194444444444, 3....",TASSO,Bona Soua,NaN,3.30642,9.73383,NaN,NaN
50,KOUSSOUKOU,"([KRIKOUBE, GOURE BANGUI, KOUSSOUKOU 1, KRIKOU...",NIKKI,KOUSSOUKOU 1,NaN,3.24281,9.98464,NaN,NaN
84,TOTOROU,"([TOTOROU], [3.2006555555555556], [9.951266666...",NIKKI,TOTOROU,NaN,3.20066,9.95127,NaN,NaN
89,YAO-GOUROU,"([Kinta, Gah baka, Gnelkourarou, Bonyakou], [3...",SEREKALE,Gnelkourarou,NaN,3.02861,10.0606,NaN,NaN
90,YORAROU,"([Séré Gando, Yorarou, Yorarou, Dakarou, Damon...",SEREKALE,Yorarou,NaN,3.04201,9.91689,NaN,NaN


#All the village list in this picture

<a href="https://drive.google.com/open?id=1W2HTUI9EqR_WdigA1m-tfFJ5NryyzqeS" target="_parent"><img src="http://icons.iconarchive.com/icons/marcus-roberto/google-play/256/Google-Drive-icon.png" alt="Open In Drive"/></a>

In [119]:
df_villages_loc_pumps = df_villages_loc_pumps[['village',0,'Arrondissement','Name','Latitude','Longitude']]

In [120]:
df_villages_loc_pumps.columns=['village','pump_loc','arrondissement','name','latitude','longitude']

# LOCALITES

In [122]:
df_2009.columns

Index(['arrondissement', 'village', 'localite', 'population', 'type', 'etat',
       'longitude', 'latitude', 'domaine', 'latitude_decimals',
       'longitude_decimals'],
      dtype='object')

In [123]:
df_loc_2009 = df_2009.groupby('localite').agg(
                            {
                            "type":'unique',
                            "population":'max',
                            "latitude_decimals":'mean',
                            "longitude_decimals":'mean',
                            "arrondissement":'unique',
                            "village":'unique'})

In [124]:
df_loc_2009.columns = [col for col in df_loc_2009.columns.values]

In [125]:
df_loc_2009=df_loc_2009.reset_index()

In [126]:
df_loc_2009[['village','arrondissement']]=df_loc_2009.apply(lambda x: pd.Series([x['village'][0],
                                                                       x['arrondissement'][0]]),axis=1)

In [127]:
df_loc_2009.population=pd.to_numeric(df_loc_2009.population, errors='coerce')

In [130]:
def count_pm_fpm(x):
    types_ouvrage=list(x["type"])
    pm=types_ouvrage.count('PM')
    fpm=types_ouvrage.count('FPM')
    pt=types_ouvrage.count('PT')
    none = types_ouvrage.count('/')
    
    if(len(types_ouvrage)!= (pm+fpm+pt+none)):
        print(types_ouvrage)
    return [pm,fpm,pt]

In [131]:
df_loc_2009[['PM','FPM','PT']]=df_loc_2009.apply(lambda x: pd.Series(count_pm_fpm(x)), axis=1)

['FPM', 'PT', 'PM', 'BF 1', 'BF 2', 'BF 3', 'BF 4', 'BF 5']
['PEA (Rampes de 4 robinets)', 'FPM']


In [132]:
df_loc_2009=df_loc_2009.drop("type",axis=1)

In [133]:
df_loc_2009['fpm_lisette']=df_loc_2009['FPM']

In [135]:
df_loc_2009['localite_lisette']=df_loc_2009['localite']

In [136]:
l_villages = df_villages_lisette['village'].to_list()
df_loc_2009[['village_lisette','vil_match']]=df_loc_2009.apply(
    lambda x: pd.Series([(a[0],a[1]) for a in process.extract(x['village'], 
                                                                 l_villages, 
                                                                 limit=1)][0]), axis=1)


In [141]:
i=0
locs_2009 = [unidecode.unidecode(loc.lower()) for loc in df_loc_2009['localite'].to_list()]

for loc_lisette,vill_lisette,arron_lisette in zip(df_villages_loc_pumps['pump_loc'].to_list(),
                                                  df_villages_loc_pumps['village'].to_list(),
                                                  df_villages_loc_pumps['arrondissement'].to_list()):
    for loc in list(set(loc_lisette[0])):
        find_loc = process.extract(unidecode.unidecode(loc.lower()),locs_2009,limit=1)
        i+=1
        loc_2009 = find_loc[0][0]
        match =find_loc[0][1]
        loc_f=process.extract(loc_2009,df_loc_2009['localite'].to_list(),limit=1)[0][0]
        ind=df_loc_2009[df_loc_2009['localite']==loc_f].index[0]
        if find_loc[0][1]<88:
            print(loc,loc_2009,match)
            df_loc_2009.loc[ind,'fpm_lisette']= None
            df_loc_2009.loc[ind,'localite_lisette']= None
            df_loc_2009.loc[ind,'arron_lisette']= None
        else:
            df_loc_2009.loc[ind,'fpm_lisette']=int(loc_lisette[0].count(loc))
            df_loc_2009.loc[ind,'localite_lisette']=loc
            df_loc_2009.loc[ind,'village_lisette']= vill_lisette
            df_loc_2009.loc[ind,'arron_lisette']= arron_lisette
            
            
print('nº of localités %3d'%i)

SEMEROU serou 83
CAMP goure tchame 68
MADINA goure waignan 60
DANNON bannon 83
GOURE GOUDA gossogui goure
baaba 86
GANCHON gnanhoun 80
BOUDAROU bouerou 80
Sidi kparou agbo barthelemy kparou 86
Gourè Alidou gannoure halidou 86
Gnelsanda gnel sanna 84
Gneltoko looko 72
Sèriaworou serewondirou 73
Banikanni baani 72
Gourè Kariya goure karea 87
Aladji Gaani Kparou alpha kparou 86
GREFE KPAROU agbo barthelemy kparou 86
Ouenra 2 angankirou 2 86
Ouenra 1 angankirou 1 86
SACABANSSI sakabansi 84
SACABANSSI MISSION saabo 72
Baraworou saka worou 74
Dadi kparou agbo barthelemy kparou 86
Gandérou dandegourou 74
SOUNAAROU soumarou 82
DJIBRIL KPAROU agbo barthelemy kparou 86
TOTOROU tontarou 80
Gah Sounon Kparou alazi kparou (ferme
general) 86
Petit paris tepa 68
Dambourasson tepa
(danbourasson; tairou kparou) 83
nº of localités 280


In [143]:
df_villages_loc_pumps[df_villages_loc_pumps['village']=='TANAKPE']['arrondissement'].values[0]

'TASSO'

In [144]:
df_loc_2009['arron_lisette']=df_loc_2009.apply(lambda x: df_villages_loc_pumps[
                                                        df_villages_loc_pumps['village']==x['village_lisette']
                                                                ]['arrondissement'].values[0],
                                              axis=1)

In [145]:
df_loc_2009[df_loc_2009['fpm_lisette'].isna()]

,localite,population,latitude_decimals,longitude_decimals,arrondissement,village,PM,FPM,PT,fpm_lisette,localite_lisette,village_lisette,vil_match,arron_lisette
2,Agbo Barthélémy Kparou,100.0,NaN,NaN,NIKKI,Gori,0,0,0,NaN,None,GORI,100,NIKKI
4,Alazi Kparou (Ferme\nGénéral),22.0,10.028500,3.337306,NIKKI,Sakabansi,0,1,0,NaN,None,SAKABANSI,100,NIKKI
5,Alpha Kparou,73.0,9.891278,2.811806,BIRO,Ourarou,0,1,0,NaN,None,OURAROU,100,BIRO
7,Angankirou 1,283.0,9.796639,3.303056,NIKKI,Tontarou,1,0,0,NaN,None,ANGANKIROU,100,NIKKI
8,Angankirou 2,565.0,NaN,NaN,NIKKI,Tontarou,0,0,0,NaN,None,ANGANKIROU,100,NIKKI
11,Baani,600.0,10.003000,3.129694,SEREKALE,Ganrou Peuhl,0,1,0,NaN,None,BAANI,90,SEREKALE
112,Gannourè Halidou,150.0,NaN,NaN,OUENOU,Lafiarou,0,0,0,NaN,None,ALAFIAROU,94,OUENOU
137,Gnel Sanna,1402.0,9.941889,3.446014,OUENOU,Sansi,1,1,0,NaN,None,SANSI,100,OUENOU
165,Gourè Karèa,160.0,NaN,NaN,GNONKOUROKALI,Guèma,0,0,0,NaN,None,GUEMA,89,GNONKOUROKALI
226,Looko,60.0,NaN,NaN,NIKKI,Sakabansi,0,0,0,NaN,None,SAKABANSI,100,NIKKI


In [146]:
df_loc_2009['fpm']=df_loc_2009.apply(lambda x: x['fpm_lisette'] if x['fpm_lisette'] else x['FPM'],
                                         axis=1)

In [148]:
df_loc_2009['localite']=df_loc_2009.apply(lambda x: x['localite_lisette'] if x['localite_lisette'] else x['localite'],
                                         axis=1)

In [149]:
len(df_villages_loc_pumps['village'].unique())

90

In [150]:
len(list(df_loc_2009['village_lisette'].unique()))

88

In [151]:
len(df_loc_2009['village'].unique())

56

In [153]:
df_pop_vil=df_loc_2009.groupby('village_lisette').agg({'population':'sum'})

In [154]:
df_pop_vil.columns = [col for col in df_pop_vil.columns.values]

In [155]:
df_pop_vil=df_pop_vil.reset_index()

In [156]:
df_pop_vil.columns=['village','population']

In [157]:
df_vil_pre=df_villages_loc_pumps[['village','arrondissement','latitude','longitude']]

In [158]:
df_vil_final=pd.merge(df_vil_pre,df_pop_vil,on='village',how='outer')

In [159]:
df_vil_final.groupby('arrondissement').sum()['population']

arrondissement
BIRO             14115.0
GNONKOUROKALI    23574.0
NIKKI            45664.0
OUENOU           18334.0
SEREKALE         13000.0
SUYA              7556.0
TASSO            15268.0
Name: population, dtype: float64

In [160]:
## FINAL VILLAGE LIST

df_vil_final.head()

,village,arrondissement,latitude,longitude,population
0,ALAFIAROU,OUENOU,9.9058555,3.4009337,1550.0
1,ANGANKIROU,NIKKI,9.8002871,3.3042026,1098.0
2,BAANI,SEREKALE,10.0136933,3.1317472,800.0
3,BANTERE,SUYA,9.853102,3.0919219,500.0
4,BARKEDJE,OUENOU,9.8537362,3.3813429,516.0


In [161]:
df_vil_final.columns=['Name','District','Latitude','Longitude','Population']

In [163]:
df_local_final=df_loc_2009[['localite','arron_lisette','village_lisette','population',
                            'latitude_decimals','longitude_decimals']]
df_local_final.columns=['localite','arrondissement','village','population','latitude','longitude']


In [164]:
df_loc_2009['n_per/fpm'] = df_loc_2009.apply(
                                    lambda x: 10000 if x['fpm']==0 else x['population']/x['fpm'],axis=1)


In [165]:
def status(x):
    ind=x['n_per/fpm']
    status='unknown'
    if np.isnan(ind) or isinstance(ind, str):
        return status
    elif ind<250:
        status='green'
    elif ind <500:
        status='orange'
    elif ind==10000:
        status='super_red'
    elif ind>500:
        status='red'
    return status

In [166]:
df_loc_2009['status'] = df_loc_2009.apply(lambda x: status(x),axis=1)

In [168]:
df_loc_2009_final=df_loc_2009[['localite','village_lisette','arron_lisette','population',
                  'latitude_decimals','longitude_decimals',
                  'PM','PT','fpm','n_per/fpm','status']]

In [179]:
df_loc_2009_final.columns=['localite','village','arrondissement','population',
                  'latitude','longitude',
                  'pm','pt','fpm','n_per/fpm','status']


In [170]:
##final localites list
df_loc_final = df_loc_2009_final[['localite','village','population',
                  'latitude','longitude']]

In [589]:
df_loc_final.columns=['Name','Village','Population',
                  'Latitude','Longitude']

# JOIN final and 2016

In [182]:
df_loc_2009_final.sort_values(by=['arrondissement','village'])

,localite,village,arrondissement,population,latitude,longitude,pm,pt,fpm,n_per/fpm,status
15,Bah Boré,BIRO,BIRO,150.0,NaN,NaN,0,0,0.0,10000.000000,super_red
28,BIRO NOUKOUROU,BIRO,BIRO,5000.0,9.903519,2.945014,1,1,4.0,1250.000000,red
117,Gaouré (Sanni\nKparou),BIRO,BIRO,120.0,NaN,NaN,0,0,0.0,10000.000000,super_red
241,Oumarou Kparou 1 (Sabo),BIRO,BIRO,200.0,NaN,NaN,0,0,0.0,10000.000000,super_red
262,SARAVONDO,BIRO,BIRO,350.0,9.904556,2.974125,1,0,1.0,350.000000,orange
...,...,...,...,...,...,...,...,...,...,...,...
199,Kinta,YAO-GOUROU,NaN,300.0,10.037319,3.036597,1,0,1.0,300.000000,orange
57,Dakarou,YORAROU,NaN,300.0,9.896528,3.029972,0,0,1.0,300.000000,orange
59,Damonsi,YORAROU,NaN,250.0,NaN,NaN,0,0,1.0,250.000000,orange
100,Yorarou,YORAROU,NaN,100.0,NaN,NaN,0,0,3.0,33.333333,green


In [176]:
df_2016_tomerge

,arrondissement,village,localite,hameau,etat,redevance,annee,marque
0,BIRO,Biro,Nallou,Nallou Banikparou,Fonctionnel,Jamais,2004.0,Vergnet
1,BIRO,Biro,Goutto,Goutto,Fonctionnel,Jamais,2011.0,Vergnet
2,BIRO,Biro,Nanlou,Nanlou,Fonctionnel,Jamais,NaN,Vergnet
3,BIRO,Biro,Sarawodo,Sarawodo,Fonctionnel,Parfait,NaN,Vergnet
4,BIRO,Biro,Biro,Biro 1,Fonctionnel,Parfait,NaN,Vergnet
...,...,...,...,...,...,...,...,...
348,TASSO,Tanakpé,Sèkentè,Sèkentè,Panne,Jamais,NaN,Vergnet
349,TASSO,Chein,Saka Worou,Saka Worou,Fonctionnel,Une fois,NaN,Vergnet
350,TASSO,Tchein,Chein Gbérou Kpanin,Chein Kpa (Tchein Kala),Fonctionnel,Jamais,NaN,Vergnet
351,TASSO,Chein,Chein Gourou,Chein Gourou,Fonctionnel,Jamais,NaN,Vergnet


In [237]:
df_g_local_2016=df_2016_tomerge.groupby('localite').agg({'hameau':'unique',
                                         'localite':'count'})#.reset_index()

In [238]:
df_g_local_2016.columns

Index(['hameau', 'localite'], dtype='object')

In [239]:
df_g_local_2016.columns=['hameau','fpm_2016']

In [240]:
df_g_local_2016=df_g_local_2016.reset_index()

In [243]:
df_g_local_2016.fpm_2016.sum()

353

In [244]:
len(df_g_local_2016)

229

In [245]:
df_loc_2009_final.shape

(326, 12)

In [246]:
l_loc_ini=[unidecode.unidecode(a.lower()) for a in df_loc_2009_final.localite.to_list()]

In [247]:
df_g_local_2016[['loc','match']]=df_g_local_2016.apply(
    lambda x: pd.Series([(a[0],a[1]) 
             if a[1]>86
             else
             None
                         for a in process.extract(unidecode.unidecode(x['localite'].lower()), 
                                                                 l_loc_ini, 
                                                                 limit=1)][0]), axis=1)

In [248]:
df_loc_2009_final['loc']=df_loc_2009_final.apply(
    lambda x: unidecode.unidecode(x['localite'].lower()), axis=1
)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [249]:
df_g_local_2016

,localite,hameau,fpm_2016,loc,match
0,Alafiarou,"[Alafiarou 1, Alafiarou 2]",2,alafiarou,100.0
1,Alazi kparou,[Alazi kparou],1,alazi kparou (ferme\ngeneral),90.0
2,Angankirou1,"[Angankirou 1 (1), Angankirou 1 (2), Angankiro...",3,angankirou 1,96.0
3,Asssolè,[Assolè],1,assole,92.0
4,Baani,[Bani Peulh],2,baani,100.0
...,...,...,...,...,...
224,Yékéna,[Yékéna II],1,yekena,100.0
225,gah maro peulh,[gah maro peulh],1,gah maro peuhl,93.0
226,gnel soumanou,[Gnel soumanou],1,gnel soumanou,100.0
227,gourè élibo,[Gourè élibo],1,goure elibo,100.0


In [252]:
df_loc_1619=pd.merge(df_loc_2009_final,df_g_local_2016[['loc','hameau','fpm_2016']], how='left',on='loc')

In [266]:
df_loc_1619['fpm_final']=df_loc_1619.apply(
    lambda x: x['fpm_2016']
    if (np.isnan(x['fpm'])) | (x['fpm']<x['fpm_2016'])
    else
    x['fpm'],
    axis=1
)

In [268]:
#df_loc_1619[['village','arrondissement','localite','hameau',
             'population','latitude','longitude','pm','pt','fpm_final']].to_csv('localites_final_2016.csv')

# JOIN final and 2016

In [270]:
df_loc_1619.sort_values(by=['arrondissement','village'])

,localite,village,arrondissement,population,latitude,longitude,pm,pt,fpm,n_per/fpm,status,loc,hameau,fpm_2016,fpm_final
15,Bah Boré,BIRO,BIRO,150.0,NaN,NaN,0,0,0.0,10000.000000,super_red,bah bore,NaN,NaN,0.0
29,BIRO NOUKOUROU,BIRO,BIRO,5000.0,9.903519,2.945014,1,1,4.0,1250.000000,red,biro noukourou,"[Biro 1, Biro 2, Biro 3, Biro 4, Biro 5]",5.0,5.0
121,Gaouré (Sanni\nKparou),BIRO,BIRO,120.0,NaN,NaN,0,0,0.0,10000.000000,super_red,gaoure (sanni\nkparou),NaN,NaN,0.0
252,Oumarou Kparou 1 (Sabo),BIRO,BIRO,200.0,NaN,NaN,0,0,0.0,10000.000000,super_red,oumarou kparou 1 (sabo),NaN,NaN,0.0
275,SARAVONDO,BIRO,BIRO,350.0,9.904556,2.974125,1,0,1.0,350.000000,orange,saravondo,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Kinta,YAO-GOUROU,NaN,300.0,10.037319,3.036597,1,0,1.0,300.000000,orange,kinta,[Kinta (Yao gourou)],1.0,1.0
59,Dakarou,YORAROU,NaN,300.0,9.896528,3.029972,0,0,1.0,300.000000,orange,dakarou,[Dakarou],1.0,1.0
61,Damonsi,YORAROU,NaN,250.0,NaN,NaN,0,0,1.0,250.000000,orange,damonsi,[Damonsi],1.0,1.0
104,Yorarou,YORAROU,NaN,100.0,NaN,NaN,0,0,3.0,33.333333,green,yorarou,[Gah Yorarou],1.0,3.0


In [271]:
df_2019_tomerge

,arrondissement,village,localite,nom_local,marque,latitude_decimals,longitude_decimals
0,BIRO,BIRO,Biro,BIRO 2,Vergnet,9.900000,2.947778
1,BIRO,BIRO,Sarawondo,SARAWONDO,Vergnet,9.903667,2.974778
2,BIRO,BIRO,Nallou,NANLOU,Vergnet,9.933333,2.908389
3,BIRO,BIRO,Biro,BIRO CENTRE,NA,9.902778,2.940639
4,BIRO,BIRO,Biro,BIRO CENTRE,Afridev,9.903028,2.946639
...,...,...,...,...,...,...,...
360,TASSO,TASSO,Fo Darou,Fo-Darou 1,Afridev,9.683333,3.231528
361,TASSO,TASSO,Tasso,TASSO,Vergnet,9.675000,3.232472
362,TASSO,TASSO,Tasso Peuhl,Kertia,Vergnet,9.654917,3.236889
363,TASSO,TASSO,Kobi,KOBI,Vergnet,9.657028,3.261167


In [290]:
df_g_local_2019=df_2019_tomerge.groupby('localite').agg({'nom_local':'unique',
                                         'localite':'count'})#.reset_index()

In [291]:
df_g_local_2019.columns

Index(['nom_local', 'localite'], dtype='object')

In [292]:
df_g_local_2019.columns=['hameau','fpm_2019']

In [293]:
df_g_local_2019=df_g_local_2019.reset_index()

In [296]:
df_g_local_2019.fpm_2019.sum()

365

In [297]:
len(df_g_local_2019)

220

In [298]:
len(df_2019_tomerge)

365

In [330]:
l_loc_ini=[unidecode.unidecode(a.lower()) for a in df_loc_1619.localite.to_list()]

In [300]:
df_g_local_2019[['loc','match']]=df_g_local_2019.apply(
    lambda x: pd.Series([(a[0],a[1]) 
             if a[1]>86
             else
             None
                         for a in process.extract(unidecode.unidecode(x['localite'].lower()), 
                                                                 l_loc_ini, 
                                                                 limit=1)][0]), axis=1)

In [301]:
df_g_local_2019['loc']=df_g_local_2019.apply(
    lambda x: unidecode.unidecode(x['localite'].lower()), axis=1
)

In [340]:
process.extract(unidecode.unidecode('Diyam Yolla 1'),l_loc_ini,limit=2)

[('diyam yalla1', 88), ('booh  1', 86)]

In [306]:
df_loc_1619lis=pd.merge(df_loc_1619,df_g_local_2019[['loc','hameau','fpm_2019']], how='left',on='loc')

In [307]:
df_loc_1619lis.head()

,localite,village,arrondissement,population,latitude,longitude,pm,pt,fpm,n_per/fpm,status,loc,hameau_x,fpm_2016,fpm_final,hameau_y,fpm_2019
0,Bakou,TANAKPE,TASSO,300.0,NaN,NaN,0,0,1.0,300.0,orange,bakou,NaN,NaN,1.0,NaN,NaN
1,Ada Gazé Kparou,SANSI,OUENOU,80.0,NaN,NaN,0,0,0.0,10000.0,super_red,ada gaze kparou,NaN,NaN,0.0,NaN,NaN
2,Agbo Barthélémy Kparou,GORI,NIKKI,100.0,NaN,NaN,0,0,NaN,NaN,unknown,agbo barthelemy kparou,NaN,NaN,NaN,NaN,NaN
3,Alafiarou,GANROU -PEULH,SEREKALE,343.0,10.06475,3.082236,1,0,2.0,171.5,green,alafiarou,"[Alafiarou 1, Alafiarou 2]",2.0,2.0,"[ALAFIAROU 2, Alafiarou 1]",2.0
4,Alazi Kparou (Ferme\nGénéral),SAKABANSI,NIKKI,22.0,10.02850,3.337306,0,0,NaN,NaN,unknown,alazi kparou (ferme\ngeneral),[Alazi kparou],1.0,1.0,NaN,NaN


In [316]:
df_loc_1619lis['fpm_final']=df_loc_1619lis.apply(
    lambda x: x['fpm_2019']
    if (np.isnan(x['fpm_final'])) | (x['fpm_final']<x['fpm_2019'])
    else
    x['fpm_final'],
    axis=1
)

In [317]:
df_loc_1619lis[['village','arrondissement','localite','hameau_x','hameau_y',
             'population','latitude','longitude','pm','pt','fpm_final']].to_csv('localites_final_2016_lis_2019.csv')

In [529]:
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

## LOCALITES 2009

In [589]:
dfbase=pd.DataFrame()
dfbase['loc_2009']=df_2009['localite'].unique().tolist()

## merge with localites 2016

In [656]:
str2Match = df_2016['localite'].unique().tolist()
strOptions = df_2009['localite'].unique().tolist()

In [657]:
name_match,ratio_match=checker(strOptions,str2Match)

In [658]:
df1 = pd.DataFrame()
df1['loc_2009']=pd.Series(strOptions)
df1['loc_2016']=pd.Series(name_match)
df1['2009_vs_2016']=pd.to_numeric(pd.Series(ratio_match),errors='coerce')

In [659]:
df_1=pd.merge(dfbase,df1,on='loc_2009',how='left')

In [661]:
str2Match = df_2019['localite'].unique().tolist()

In [662]:
name_match,ratio_match=checker(strOptions,str2Match)

In [663]:
df2 = pd.DataFrame()
df2['loc_2009']=pd.Series(strOptions)
df2['loc_2019']=pd.Series(name_match)
df2['2009_vs_2019']=pd.Series(ratio_match)

In [664]:
df_2=pd.merge(df_1,df2,on='loc_2009',how='left')

In [665]:
str2Match = df_lisette['localite'].unique().tolist()

In [666]:
name_match,ratio_match=checker(strOptions,str2Match)

In [667]:
df3 = pd.DataFrame()
df3['loc_2009']=pd.Series(strOptions)
df3['loc_lisette']=pd.Series(name_match)
df3['2009_vs_lisette']=pd.Series(ratio_match)

In [668]:
df_fin=pd.merge(df_2,df3,on='loc_2009',how='left')

## Merge datapoints of pumps

In [670]:
df_fin

,loc_2009,loc_2016,2009_vs_2016,loc_2019,2009_vs_2019,loc_lisette,2009_vs_lisette
0,Bah Boré,Tèbo Gbaba,62,Booré 2,62,Booré 1,62
1,Biro,Biro,100,Biro,100,BIRO NOUKOUROU,100
2,Gaouré (Sanni\nKparou),Sani Kao Kparou,85,Sani Kao Kparou,85,SANI KAO KPAROU,85
3,Goutto,Goutto,100,Goutto,100,GOUTTO,100
4,Massiagourou,Massiagourou,100,Massiagourou,100,MASSIAGOUROU,100
...,...,...,...,...,...,...,...
321,Fo Darou,Fo-Darou,100,Fo Darou,100,Fô darou,93
322,Sinagourou\nFrontière,Sinagourou,100,Sinagourou,100,Sinagourou frontière,100
323,Sombiri (Kertia 2),Kpiré (2),45,Soria Kpaa,54,Sombiri,100
324,Tasso,Tasso,100,Tasso,100,Tasso,100


In [680]:
df_2009_loc_fpm=df_2009.groupby('localite').agg(
            {
                'population':'mean',
                'type':'unique'
            }
                                            ).reset_index()
    

In [682]:
df_2009_loc_fpm['fpm_2009']=df_2009_loc_fpm.apply(lambda x: list(x['type']).count('FPM'),axis=1)

In [696]:
df_fin9=pd.merge(df_fin,
                 df_2009_loc_fpm[['population','localite','fpm_2009']],
                 left_on='loc_2009',
                 right_on='localite',
                 how='left').drop('localite',axis=1)


Index(['n', 'arrondissement', 'village', 'localite', 'hameau', 'etat',
       'redevance', 'annee', 'marque'],
      dtype='object')

In [699]:
df_2016_loc_fpm=df_2016.groupby('localite').agg(
            {
                'village':'count'
            }
                                            ).reset_index()
    

In [701]:
df_2016_loc_fpm.columns=['localite','fpm_2016']

In [702]:
df_fin9=pd.merge(df_fin9,
                 df_2016_loc_fpm[['localite','fpm_2016']],
                 left_on='loc_2016',
                 right_on='localite',
                 how='left').drop('localite',axis=1)


In [705]:
df_2019_loc_fpm=df_2019.groupby('localite').agg(
            {
                'village':'count'
            }
                                            ).reset_index()
    

In [706]:
df_2019_loc_fpm.columns=['localite','fpm_2019']

In [708]:
df_fin9=pd.merge(df_fin9,
                 df_2019_loc_fpm[['localite','fpm_2019']],
                 left_on='loc_2019',
                 right_on='localite',
                 how='left').drop('localite',axis=1)


In [710]:
df_lisette_loc_fpm=df_lisette.groupby('localite').agg(
            {
                'village':'count'
            }
                                            ).reset_index()
    

In [711]:
df_lisette_loc_fpm.columns=['localite','fpm_lisette']

In [712]:
df_fin9=pd.merge(df_fin9,
                 df_lisette_loc_fpm[['localite','fpm_lisette']],
                 left_on='loc_lisette',
                 right_on='localite',
                 how='left').drop('localite',axis=1)


In [713]:
df_fin9

,loc_2009,loc_2016,2009_vs_2016,loc_2019,2009_vs_2019,loc_lisette,2009_vs_lisette,population,fpm_2009,fpm_2016,fpm_2019,fpm_lisette
0,Bah Boré,Tèbo Gbaba,62,Booré 2,62,Booré 1,62,150.0,0,1,1,1
1,Biro,Biro,100,Biro,100,BIRO NOUKOUROU,100,5000.0,1,5,5,4
2,Gaouré (Sanni\nKparou),Sani Kao Kparou,85,Sani Kao Kparou,85,SANI KAO KPAROU,85,120.0,0,1,1,1
3,Goutto,Goutto,100,Goutto,100,GOUTTO,100,300.0,1,1,1,1
4,Massiagourou,Massiagourou,100,Massiagourou,100,MASSIAGOUROU,100,621.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
321,Fo Darou,Fo-Darou,100,Fo Darou,100,Fô darou,93,700.0,1,2,1,2
322,Sinagourou\nFrontière,Sinagourou,100,Sinagourou,100,Sinagourou frontière,100,250.0,0,3,2,1
323,Sombiri (Kertia 2),Kpiré (2),45,Soria Kpaa,54,Sombiri,100,200.0,0,1,1,1
324,Tasso,Tasso,100,Tasso,100,Tasso,100,850.0,1,3,2,4


In [714]:
df_fin9.to_csv('proposed_merged_localites.csv')